In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import vnstock
from datetime import datetime

In [ ]:
start_date = "2016-01-01"
end_date = "2023-08-18"
start_datetime = datetime.strptime(start_date, '%Y-%m-%d')
end_datetime = datetime.strptime(end_date, '%Y-%m-%d')
duration = (end_datetime - start_datetime).days
df = vnstock.stock_historical_data("ACB", start_date, end_date, "1D", 'stock')
df.head()

In [ ]:
features = ['close']
df = df[features]
df.rename(columns={'close': 'price'}, inplace=True)
df.head()

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df['price'])
plt.grid(True)
plt.show()

In [24]:
def MA_value(day, index, df):
    MA_v = 0
    for i in range(index-day+1, index+1):
        MA_v = MA_v + df['price'][i]
    MA_v = MA_v/day
    return MA_v

In [ ]:
MA_day_list = [5, 10, 20, 50, 200]
for MA_day in MA_day_list:
    df['MA'+str(MA_day)] = ""
    for index in range(len(df)):
        if index < MA_day:
            df['MA'+str(MA_day)][index] = 0
        else:
            df['MA'+str(MA_day)][index] = MA_value(MA_day, index, df)
    

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df['price'][1800:], label='price', linewidth=3)
for MA_day in MA_day_list:
    plt.plot(df['MA'+str(MA_day)][1800:], label='MA'+str(MA_day))
plt.grid(True)
plt.legend()
plt.show()

In [27]:
def check_condition_valid(conditions, row):
    for condition in conditions:
        if row['MA'+str(condition)] == 0:
            return False
    return True

def check_conditions_actived(conditions, row):
    for condition in conditions:
        if row['price'] < row['MA'+str(condition)]:
            return False
    return True

def check_conditions_unactived(conditions, row):
    for condition in conditions:
        if row['price'] > row['MA'+str(condition)]:
            return False
    return True

In [28]:
command = pd.DataFrame({'buy_index':[], 'buy_price':[], 'sell_index':[], 'sell_price':[], 'profit_(%)':[], 'holding_days':[]})

In [ ]:
conditions = [5, 10, 20, 50, 200]
for index, row in df.iterrows():
    if check_condition_valid(conditions, row) == False:
        continue
    elif check_conditions_actived(conditions, row):
        new_command = {'buy_index': index, 'buy_price': row['price']}
        command = pd.concat([command, pd.DataFrame([new_command])], ignore_index=True)
command.head()

In [ ]:
command_index = 0
while command_index < len(command):
    for i in range(int(command.iloc[command_index]['buy_index']), len(df)):
        if check_conditions_unactived(conditions, df.iloc[i]):
            buy_price = command.iloc[command_index]['buy_price']
            sell_price = df.iloc[i]['price']
            command.loc[command_index, 'sell_index'] = i
            command.loc[command_index, 'sell_price'] = sell_price
            command.loc[command_index, 'profit_(%)'] = round((sell_price-buy_price)/buy_price*100, 2)
            command.loc[command_index, 'holding_days'] = i - command.iloc[command_index]['buy_index']
            break
    command_index = command_index + 1
command.head(10)

In [ ]:
command = command.drop(command[command.duplicated(subset=['sell_index'])].index)
command = command.reset_index(drop=True)
command.head(20)

In [ ]:
last_command_index = len(command) - 1 # we can't use -1 because it will create a new command with index -1
if pd.isna(command.iloc[last_command_index]['sell_index']):
    command.loc[last_command_index, 'sell_index'] = len(df) - 1
    command.loc[last_command_index, 'sell_price'] = df.iloc[-1]['price']
    command.loc[last_command_index, 'profit_(%)'] = round((df.iloc[-1]['price']-command.iloc[last_command_index]['buy_price'])/command.iloc[last_command_index]['buy_price']*100, 2)
    command.loc[last_command_index, 'holding_days'] = len(df) - 1 - command.iloc[-1]['buy_index']
command.tail()

In [ ]:
df.loc[1561:1566]

In [ ]:
total_command = len(command)
total_holding_days = int(command['holding_days'].sum())
average_holding_days = round(total_holding_days/total_command, 2)
total_profit = round(command['profit_(%)'].sum(), 2)
average_profit = round(total_profit/total_command, 2)
profit_per_year = round(total_profit/(duration/365), 2)

print("Total of commands: ", total_command, "commands")
print("Total of holding days: ", total_holding_days, "days")
print("Average number of days holding a command: ", average_holding_days, "days")
print("Total profit (%): ", total_profit, "%")
print("Average profit (%) of a command: ", average_profit, "%")
print("Profit (%) per year: ", profit_per_year, '% per year')